In [272]:
import pandas as pd
import math

In [311]:
df_salaries_ca = pd.read_csv('../../../data/raw_data/salaries_ca_info.csv')
df_salaries_ca = df_salaries_ca.drop(['Unnamed: 0'], axis=1)

In [312]:
df_salaries_us = pd.read_csv('../../../data/raw_data/salaries_us_info.csv')
df_salaries_us = df_salaries_us.drop(['Unnamed: 0'], axis=1)

In [317]:
# Merge all Canada job descriptions
job_desc_ca = job_desc_ca_all.append(job_desc_ca_jr, sort=True)
job_desc_ca = job_desc_ca.append(job_desc_ca_sr, sort=True)

# Drop nans and duplicates
job_desc_ca = job_desc_ca.dropna()
job_desc_ca = job_desc_ca.drop_duplicates(subset=['Company', 'Position'])

In [318]:
def get_salary_freq(salary):
    freq = ''
    is_hourly_1 = salary.find("/hr")
    is_hourly_2 = salary.find("hourly")
    is_monthly = salary.find("/mo")
    is_yearly = salary.find("/yr")
    if (is_hourly_1 != -1) or (is_hourly_2 != -1):
        freq = 'hourly'
    elif is_monthly != -1:
        freq = 'monthly'
    elif is_yearly != -1:
        freq = 'yearly'
    else:
        freq = 'yearly'
    return freq

In [319]:
def clean_salary(salary):
    if type(salary) != float:
        salary = salary.replace("CA", "")
        salary = salary.replace("€", "")
        salary = salary.replace("₹", "")
        salary = salary.replace("$", "")
        salary = salary.replace("£", "")
        salary = salary.replace("₪", "")
        salary = salary.replace("K", "000")
        salary = salary.replace(",", "")
        is_range = salary.find("-")
        is_hourly_1 = salary.find("/hr")
        salary = salary.replace("/hr", "")
        is_hourly_2 = salary.find("hourly")
        salary = salary.replace("hourly", "")
        is_monthly = salary.find("/mo")
        salary = salary.replace("/mo", "")
        is_yearly = salary.find("/yr")
        salary = salary.replace("/yr", "")
        if is_range != -1:
            salary = salary.replace("A", "")
            salary = salary.replace("R", "")
            lower = int(salary[:is_range].replace("-", ""))
            upper = int(salary[is_range + 1:])
            salary = (lower + upper) / 2
        if (is_hourly_1 != -1) or (is_hourly_2 != -1):
            salary = int(salary) * 40 * 52
        if (is_monthly != -1):
            salary = int(salary) * 12
        if (is_yearly != -1):
            salary = int(salary)
        else:
            try:
                salary = int(salary)
            except:
                salary = float('nan')
    return salary

In [320]:
def convert_to_cad(usd):
    try:
        return usd * 1.31
    except:
        return float('nan')

In [321]:
def clean_additional_salary_info_column(df, column, hourly_mask, monthly_mask):
    df[column] = df[column].map(clean_salary)
    hourly_mask = (df['freq'] == 'hourly')
    monthly_mask = (df['freq'] == 'monthly')
    df[column][hourly_mask] *= 40 * 52
    df[column][monthly_mask] *= 12

In [322]:
df_salaries_ca['freq'] = df_salaries_ca['salary'].map(get_salary_freq)
df_salaries_ca['salary'] = df_salaries_ca['salary'].map(clean_salary)

In [323]:
hourly_mask = (df_salaries_ca['freq'] == 'hourly')
monthly_mask = (df_salaries_ca['freq'] == 'monthly')
clean_additional_salary_info_column(df_salaries_ca, 'lower_bound_salary', hourly_mask, monthly_mask)
clean_additional_salary_info_column(df_salaries_ca, 'upper_bound_salary', hourly_mask, monthly_mask)
clean_additional_salary_info_column(df_salaries_ca, 'base_pay', hourly_mask, monthly_mask)
clean_additional_salary_info_column(df_salaries_ca, 'bonus_pay', hourly_mask, monthly_mask)
df_salaries_ca['cash_bonus'] = df_salaries_ca['cash_bonus'].map(clean_salary)
df_salaries_ca['stock_bonus'] = df_salaries_ca['stock_bonus'].map(clean_salary)

/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is tr

In [324]:
df_salaries_us['freq'] = df_salaries_us['salary'].map(get_salary_freq)
df_salaries_us['salary'] = df_salaries_us['salary'].map(clean_salary).map(convert_to_cad)

In [325]:
hourly_mask = (df_salaries_us['freq'] == 'hourly')
monthly_mask = (df_salaries_us['freq'] == 'monthly')
clean_additional_salary_info_column(df_salaries_us, 'lower_bound_salary', hourly_mask, monthly_mask)
df_salaries_us['lower_bound_salary'] = df_salaries_us['lower_bound_salary'].map(convert_to_cad)
clean_additional_salary_info_column(df_salaries_us, 'upper_bound_salary', hourly_mask, monthly_mask)
df_salaries_us['upper_bound_salary'] = df_salaries_us['upper_bound_salary'].map(convert_to_cad)

clean_additional_salary_info_column(df_salaries_us, 'base_pay', hourly_mask, monthly_mask)
df_salaries_us['base_pay'] = df_salaries_us['base_pay'].map(convert_to_cad)
clean_additional_salary_info_column(df_salaries_us, 'bonus_pay', hourly_mask, monthly_mask)
df_salaries_us['bonus_pay'] = df_salaries_us['bonus_pay'].map(convert_to_cad)

df_salaries_us['cash_bonus'] = df_salaries_us['cash_bonus'].map(clean_salary).map(convert_to_cad)
df_salaries_us['stock_bonus'] = df_salaries_us['stock_bonus'].map(clean_salary).map(convert_to_cad)

/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sashananda/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
